In [556]:
from lo.sdk.api.acquisition.io.open import open
from lo.sdk.api.acquisition.data.decode import SpectralDecoder
from lo.sdk.api.acquisition.data.formats import LORAWtoRGB8

import glob
import matplotlib.pyplot as plt
import numpy as np
import os

import cv2 as cv

In [557]:
def load_image(index):
    global img, hypercube, scene_view_shape, image
    cv.destroyAllWindows()
    image = images[index]
    print(image)
    hypercube = np.load(image)
    lofile = image.split('-')
    lofile.pop()
    lofile = '-'.join(lofile) + '.loraw'
    calibration_files = glob.glob('Spectral_images/' + 'calibration*.loraw')
    factory_cali = os.path.expanduser('LO1_calibration_files/8_5mm')
    decode = SpectralDecoder.from_calibration(factory_cali, calibration_files[0])
    with open(lofile) as f:
        for frame in f:
            metadata, scene_view, spectra = decode(frame, scene_decoder=LORAWtoRGB8)
            scene_view_shape = scene_view.shape[:2]
            img = scene_view
            slice_image = hypercube[:,:,95]
            slice_image = cv.normalize(slice_image, None, 0, 255, cv.NORM_MINMAX).astype(np.uint8)
            slice_image = cv.resize(slice_image, (scene_view_shape[1]-565, scene_view_shape[0]-140), interpolation=cv.INTER_LINEAR)
            slice_image_shape = slice_image.shape[:2]
            desired_width = scene_view_shape[1]
            desired_height = scene_view_shape[0]
            top = (desired_height - slice_image_shape[0]) // 2
            bottom = desired_height - slice_image_shape[0] - top
            left = (desired_width - slice_image_shape[1]) // 2
            right = desired_width - slice_image_shape[1] - left

            # Add black border to the image
            slice_image = cv.copyMakeBorder(slice_image,top-20,bottom,left-40,right,cv.BORDER_CONSTANT,value=(0, 0, 0)  # Color of the border (black)
                                             )


            cv.imshow('Select Corners', scene_view)
            cv.setMouseCallback('Select Corners', select_point)



            #slice_image = cv.resize(slice_image, (2000, 2000), interpolation=cv.INTER_LINEAR)
            cv.imshow('Spectral_image', slice_image)
            break
    

In [558]:
def select_point(event, x, y, flags, param):
    """Handle mouse events to select points."""
    global points

    if event == cv.EVENT_LBUTTONDOWN:
        points.append((x, y))
        cv.circle(img, (x, y), 5, (0, 0, 255), -1)  # Red dot for points
        cv.imshow('Select Corners', img)

        if len(points) % 4 == 0:
            draw_square()
            warp_and_resize_image()

In [559]:
def draw_square():
    """Draw lines between the selected points."""
    for i in range(len(points)):
        cv.line(img, points[i], points[(i + 1) % 4], (0, 255, 0), 1)  # Green lines
    cv.imshow('Select Corners', img)

In [560]:
def warp_and_resize_image():
    """Apply perspective transform and resize the image."""
    channels = hypercube.shape[2]
    final_hypercube = np.zeros((scene_view_shape[1]-565, scene_view_shape[0]-140, channels))
    for channel in range(channels):
        slice_image = hypercube[:,:,channel]
        slice_image = cv.normalize(slice_image, None, 0, 255, cv.NORM_MINMAX).astype(np.uint8)
        slice_image = cv.resize(slice_image, (scene_view_shape[1]-565, scene_view_shape[0]-140), interpolation=cv.INTER_LINEAR)
        slice_image_shape = slice_image.shape[:2]
        desired_width = scene_view_shape[1]
        desired_height = scene_view_shape[0]
        top = (desired_height - slice_image_shape[0]) // 2
        bottom = desired_height - slice_image_shape[0] - top
        left = (desired_width - slice_image_shape[1]) // 2
        right = desired_width - slice_image_shape[1] - left

        # Add black border to the image
        slice_image = cv.copyMakeBorder(slice_image,top-20,bottom,left-40,right,cv.BORDER_CONSTANT,value=(0, 0, 0)  # Color of the border (black)
                                             )

        src_pts = np.array(points, dtype='float32')
        widthA = np.linalg.norm(np.array(src_pts[0]) - np.array(src_pts[1]))
        widthB = np.linalg.norm(np.array(src_pts[2]) - np.array(src_pts[3]))
        maxWidth = max(int(widthA), int(widthB))
        heightA = np.linalg.norm(np.array(src_pts[0]) - np.array(src_pts[3]))
        heightB = np.linalg.norm(np.array(src_pts[1]) - np.array(src_pts[2]))
        maxHeight = max(int(heightA), int(heightB))
        dst_pts = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype='float32')
        M = cv.getPerspectiveTransform(src_pts, dst_pts)
        warped = cv.warpPerspective(slice_image, M, (maxWidth, maxHeight))
        final_hypercube[:,:,channel] = warped
    cv.imshow('Resized Image', warped)
    np.save(os.path.join('Spectral_images/Warped_spectral/', image.split('/')[1].split('.')[0] + '-warped.npy'),final_hypercube)


In [561]:
def handle_key(event):
    global current_index
    if event == 2:  # Left arrow key
        if current_index > 0:
            current_index -= 1
            load_image(current_index)
    elif event == 3:  # Right arrow key
        if current_index < len(images) - 1:
            current_index += 1
            load_image(current_index)
        else:
            print("End of images")

In [562]:
images = glob.glob('Spectral_images/' + '*.npy')
current_index = 0
points = []

# Load the first image
load_image(current_index)

cv.setMouseCallback('Select Corners', select_point)

while True:
    key = cv.waitKey(0) & 0xFF
    if key == 27:  # ESC key to exit
        break
    elif key in [2, 3]:  # Arrow keys
        points = []
        handle_key(key)

cv.destroyAllWindows()
cv.waitKey(1)





    

Spectral_images/Bkbnpkq2CA-20240627-114001-108267-enhanced.npy


ValueError: could not broadcast input array from shape (1823,1836) into shape (1867,1908)

-1